Show interesting geodesic experiments here

One that is particularly interesting is the difference between the finding the geodesic of the smooth manifold from the determnistic decoder vs. noisy from probabilistic decoder. Here, I can use the same encoder but take variance into accoutn for the probabilistic one.

The probabilistic one is more faitful to the true data manifold's geometry because unobserved data is given a higher distance value. In other words, the Jacobian is higher when the variance changes from small to large, indicating a transition to a more uncertain region of the manifold. As such, curves will be stretched in these regions and geodesics are less likely to be formed.




Thoughts:

How can I get this jacobian of the variance for the VAE? Does the variance come from decomposing the initial embedding the latent space as a mean value and a variance value conditioned on the input? 

Ok, it seems that instead of viewing the decoding as a deterministic function, it needs to be seen as the probability of an output in observation space conditioned on a latent representation. Outputs from the decoder are no longer point estimates but rather probability distributions.